In [1]:
import nibabel as nib
#import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from sklearn.cluster import KMeans
from os import listdir
from os.path import join, exists
from scipy.io import savemat,loadmat

In [2]:
directory = 'OsteoarthritisInitiative/my_predictions/nii_files'
directory_org = 'OsteoarthritisInitiative/NIFTY'


# Make empty list
b = os.listdir(directory)
b = [filename[:-12] for filename in b]

df_mri = pd.read_csv("mri_data", index_col ="Unnamed: 0")

for i in set(df_mri.index): 
    if i[-2:]=="_s":
        df_mri = df_mri.drop([i])
        print("Dropped", i )

        
iindex = []
for i in b:  
    if i in df_mri.index:
        continue
        
    if i[-7:] == '_PRED_s':
        continue
    
    iindex.append(i)

if len(iindex)!=0:
    empty = pd.DataFrame(np.zeros((len(iindex),23)), columns = df_mri.columns, index = iindex)
    df_mri = df_mri.append(empty)

/tmp/ipykernel_194/108624697.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_mri = df_mri.append(empty)


## Add total amount of raw cartilage

In [3]:
# add amount of cartilage as variable 
a=0
for idx in df_mri.index:
        a +=1
        if df_mri["Cartilage: 4"].loc[idx] != 0.0:
            continue
        
        if idx[-7:] == '_PRED_s':
            continue
            
        path = directory+"/"+idx+"_PRED.nii.gz"
        img = nib.load(path).get_fdata()
        

        for i in range(1,5):
            df_mri["Cartilage: " + str(i)].loc[idx] = np.where(img==i)[0].shape[0]
        
        if a/10 == a//10:
            print(a)
            df_mri.to_csv("mri_data")

## Split Tibial and Femural compartments

In [4]:
# The Tibial compartments was segmented as a single class by the methods.
# Here, this class is split into two compartments using KMeans
# There is a low-coordinate compartment and a high-coordinate compartment.
# Outside this function, knowing whether this is a left or a right knee, these
# compartments can then be assigned as medial and lateral.

def SplitTibial(mask):
    # Input is binary mask for both tibial compartments
    [xs, ys, zs] = np.nonzero(mask)
    # Split mask in two components usin KMeans 
    coor = np.array([xs, ys, zs]).T
    model = KMeans(n_clusters=2)
    model.fit(coor)
    coor0 = coor[model.labels_==0,:]
    coor1 = coor[model.labels_==1,:]
    # Order by sagittal coordinate. 
    # Outside this function, knowing left/right, will translate this into medial/lateral
    mean0 = np.mean(coor0,axis=0)
    mean1 = np.mean(coor1,axis=0)
    if mean0[2] > mean1[2]:
        dummy = coor1
        coor1 = coor0
        coor0 = dummy
    # Compute volume for the two compartments
    vol0 = coor0.shape[0]
    vol1 = coor1.shape[0]
    # Extract ROIs for both tibial compartments. This will be used to define load-bearing part of femoral cartilage    
    roi = np.array([np.quantile(coor0,0.001,axis=0), np.quantile(coor0,0.999,axis=0), 
                    np.quantile(coor1,0.001,axis=0), np.quantile(coor1,0.999,axis=0)])
    if False:
        mi = np.min(roi,axis=0).astype('int')
        ma = np.max(roi,axis=0).astype('int')
        print('roi',roi)
        print('min',mi)
        print('max',ma)
        #showMask(mask[mi[0]:ma[0],mi[1]:ma[1],mi[2]:ma[2]])
    return vol0, vol1, roi

In [5]:
# The femoral compartment is segmented as a single class by the methods
# Here, the compartment is first split into a "medial" and a "lateral" compartment.
# Like for the tibial compartment, these are really low-coordinate and high-coordinate compartments
# that are the defined as medial and lateral outside this function. 
# This medial/lateral split is done using the bounding box coordinates from the tibial compartments
# Inbetween the tibial compartments is a gap. The central third is discarded. The rest now defines
# two compartments purely by the sagittal coordinate. 
# Each of these two compartments are then reduced to an approximation of a load-bearing region by:
#  - Center of mass (CoM) is computed
#  - Main mode of variation defines a direction from front->back but typically a bit up towards the back
#  - Everything above this line going through the CoM is discarded

from skimage.measure import label   
from sklearn.decomposition import PCA

def largestComponent(mask):
    if np.sum(mask) == 0:
        print('largestComponent found empty mask!!')
        return mask
    labels = label(mask)
    largestCC = labels == np.argmax(np.bincount(labels.flat)[1:])+1
    if False and np.sum(largestCC) < np.sum(mask):
        print('largestComponent vol decreased',np.sum(mask),np.sum(largestCC))
    return largestCC

def BelowCenter(mask, show = False):
    [xs, ys, zs] = np.nonzero(mask) # # coordinates are head->toe, front->back, left-right
    X = np.concatenate((xs.reshape(-1,1), ys.reshape(-1,1)),axis=1)
    pca = PCA(n_components=1)
    pca.fit(X)
    mainDir = pca.components_[0]
    mainDir /= np.linalg.norm(mainDir)
    upDir = np.array([mainDir[1], -mainDir[0]])
    if upDir[0]>0: # pointing down
        upDir = -upDir
    meanX = np.mean(xs) # Axial, head->toe
    meanY = np.mean(ys) # Coronal, front->back
    sz = mask.shape
    for x in range(sz[0]):
        for y in range(sz[1]):
            d = np.array([x - meanX, y - meanY])
            up = np.dot(d,upDir)
            if up>0:
                mask[x,y,:] = 0
    return mainDir, upDir, meanX, meanY  # dont need to return, mask is changed inplace

def SplitFemoral(mask, ROIs):
     # Expand medio-laterally to cover full femoral compartment - without including the anterior femoral part towards the trochlear grove
    gap = ROIs[2,2]-ROIs[1,2] # between max of first component and min of last component
    ROIs[1,2] += gap/3
    ROIs[2,2] -= gap/3
    ROIs = np.round(ROIs).astype('int')
    vols = np.zeros(2)
    # coordinates are head->toe, front->back, left-right 
    # First compartment (lowest sagittal coordinates): front to back and then until max sagittal coordinate
    submask = mask[:, :, :ROIs[1,2]]
    #if makeFigure>1:
     #   showMask(submask,'Fem 0',True)
    BelowCenter(submask)
    #if makeFigure>1:
    #   showMask(submask,'Fem 0 below',True)
    vols[0] = np.sum(submask)
    # Second compartment (highest sagittal coordinates): front to back and then from min sagittal coordinate
    submask = mask[:, :, ROIs[2,2]:]
    #if makeFigure>1: 
    #    showMask(submask,'Fem 1',True)
    BelowCenter(submask)
    #if makeFigure>1:
    #    showMask(submask,'Fem 1 below',True)
    vols[1] = np.sum(submask)
    return vols

In [6]:
df_mri= df_mri.drop(["9023407-Left-V01"])
df_mri= df_mri.drop(["9025994-Left-V06"])
df_mri= df_mri.drop(["9061827-Right-V05"])

In [7]:
# add amount of medial/lateral tibial
a=0
for idx in df_mri.index:
        a +=1
        roi = None #sequire computed for the right mask
        
        if df_mri["Lateral Femur"].loc[idx] != 0:
            continue
        
        if idx[-7:] == '_PRED_s':
            continue
        
        path = directory+"/"+idx+"_PRED.nii.gz"
        print(path)
        mask = nib.load(path).get_fdata()
        Mask_tib = np.where(mask==2,1,0)
        Mask_fem = np.where(mask==1,1,0)
        
        vol0, vol1, roi = SplitTibial(Mask_tib)
                
        df_mri.loc[idx, ("Medial Tibial")]=vol0
        df_mri.loc[idx, ("Lateral Tibial")]=vol1
        
        vol2,vol3 = SplitFemoral(Mask_fem, roi)
        df_mri.loc[idx, ("Medial Femur")]=vol2
        df_mri.loc[idx, ("Lateral Femur")]=vol3
                           
        if a/10 == a//10:
            print(a)
            df_mri.to_csv("mri_data")

# Get min dist ml. tib og fem

In [8]:
#TOY
def get_min_dist(mask1, mask2):
    
    p1 = np.argwhere(mask1!=0)
    p2 = np.argwhere(mask2!=0)
    p2_close = np.mean(p2, axis =0)
    p1_close = np.mean(p1, axis =0)
    
    d_min = np.sqrt(np.sum((p1_close-p2_close)**2, axis=0))
    
    for i in p1:
        dist_opt = np.sqrt(np.sum((i-p2_close)**2, axis=0))
        
        if dist_opt>= d_min: 
            continue
        
        elif d_min >= dist_opt:
            d_min = dist_opt
            p1_close = i
        
    for j in p2:
        dist_opt = np.sqrt(np.sum((p1_close-j)**2, axis=0))
        
        if dist_opt>= d_min: 
            continue
        
        elif d_min >= dist_opt:
            d_min = dist_opt
            p2_close = j

      
    return d_min

#get_min_dist(Mask_tib, Mask_fem)

In [9]:
# add amount of medial/lateral tibial
a=0
for idx in df_mri.index:
        a +=1
        roi = None #sequire computed for the right mask
        
        if df_mri["dist_fem_tib"].loc[idx] != 0:
            continue
            
        path = directory+"/"+idx+"_PRED.nii.gz"
        mask = nib.load(path).get_fdata()
        Mask_tib = np.where(mask==2,1,0)
        Mask_fem = np.where(mask==1,1,0)
        
        df_mri.loc[idx, "dist_fem_tib"] = get_min_dist(Mask_tib, Mask_fem)
        #print(get_min_dist(Mask_tib, Mask_fem))

        if a/10 == a//10:
            print(a)
            df_mri.to_csv("mri_data")

# Miniscus Compartment

In [10]:
def SplitMiniscus(mask):
    # Input is binary mask for both tibial compartments
    [xs, ys, zs] = np.nonzero(mask)
    # Split mask in two components usin KMeans 
    coor = np.array([xs, ys, zs]).T
    model = KMeans(n_clusters=2)
    model.fit(coor)
    coor0 = coor[model.labels_==0,:]
    coor1 = coor[model.labels_==1,:]
    # Order by sagittal coordinate. 
    # Outside this function, knowing left/right, will translate this into medial/lateral
    mean0 = np.mean(coor0,axis=0)
    mean1 = np.mean(coor1,axis=0)
    if mean0[2] > mean1[2]:
        dummy = coor1
        coor1 = coor0
        coor0 = dummy
    # Compute volume for the two compartments
    vol0 = coor0.shape[0]
    vol1 = coor1.shape[0]
    if False:
        mi = np.min(roi,axis=0).astype('int')
        ma = np.max(roi,axis=0).astype('int')
        print('min',mi)
        print('max',ma)
    return vol0, vol1

In [11]:
# add amount of medial/lateral tibial
a=0
for idx in df_mri.index:
        a +=1
        
        if df_mri["Lateral Miniscus"].loc[idx] != 0:
            continue
        
        if idx[-7:] == '_PRED_s':
            continue
            
        path = directory+"/"+idx+"_PRED.nii.gz"
        mask = nib.load(path).get_fdata()
        Mask_tib = np.where(mask==4,1,0)
        
        vol0, vol1 = SplitMiniscus(Mask_tib)
                
        df_mri.loc[idx, ("Medial Miniscus")]=vol0
        df_mri.loc[idx, ("Lateral Miniscus")]=vol1
        
                   
        if a/10 == a//10:
            print(a)
            df_mri.to_csv("mri_data")

# Intensities

In [12]:
def GetIntensities(idx):
    
    path_img = directory_org+"/"+idx+".nii.gz"
    img = nib.load(path_img).get_fdata()
    
    path = directory+"/"+idx+"_PRED.nii.gz"
    mask = nib.load(path).get_fdata()
    
    Img_intensity_mean = np.mean(img)
    Img_intensity_median = np.median(img)
    
    ix_tib = np.where(mask==2)
    int_tib_mean = np.mean(img[ix_tib])
    int_tib_median = np.median(img[ix_tib])

    ix_fem = np.where(mask==1)
    int_fem_mean = np.mean(img[ix_fem])
    int_fem_median = np.median(img[ix_fem])

    ix_min = np.where(mask==4)
    int_min_mean = np.mean(img[ix_min])
    int_min_median = np.median(img[ix_min])

    return Img_intensity_mean, Img_intensity_median, int_tib_mean, int_tib_median, int_fem_mean, int_fem_median, int_min_mean, int_min_median

#GetIntensities(idx)

In [13]:
# add amount of medial/lateral tibial
a=0
for idx in df_mri.index:
        a += 1

        if df_mri["int_min_median"].loc[idx] != 0:
            continue

        Img_intensity_mean, Img_intensity_median, int_tib_mean, int_tib_median, int_fem_mean, int_fem_median, int_min_mean, int_min_median = GetIntensities(idx)
                
        df_mri.loc[idx, ("Img_intensity_mean")] = Img_intensity_mean
        df_mri.loc[idx, ("Img_intensity_median")] = Img_intensity_median

        df_mri.loc[idx, ("int_tib_mean")] = int_tib_mean
        df_mri.loc[idx, ("int_tib_median")] = int_tib_median
        
        df_mri.loc[idx, ("int_fem_mean")] = int_fem_mean
        df_mri.loc[idx, ("int_fem_median")] = int_fem_median

        df_mri.loc[idx, ("int_min_mean")] = int_min_mean
        df_mri.loc[idx, ("int_min_median")] = int_min_median

        print("Done: ", idx)
        
        if a/10 == a//10:
            print(a)
            df_mri.to_csv("mri_data")

Done:  9985803-Right-V03
Done:  9986207-Left-V00


# Entropy

In [14]:
#Should get value between 0 and 8, 0 being no info, 8 compleately randome 
def entropy(im):    
    # Compute normalized histogram -> p(g)
    p = np.array([(im==v).sum() for v in range(256)])
    p = p/p.sum()
    
    # Compute e = -sum(p(g)*log2(p(g)))
    e = -(p[p>0]*np.log2(p[p>0])).sum()
    
    return e

In [15]:
a=0
for idx in df_mri.index:
        a += 1

        if df_mri["Entropy Meniscus"].loc[idx] != 0:
            continue

        path_img = directory_org+"/"+idx+".nii.gz"
        img = nib.load(path_img).get_fdata()

        path = directory+"/"+idx+"_PRED.nii.gz"
        mask = nib.load(path).get_fdata()

        ix_fem = np.where(mask==1)
        ix_tib = np.where(mask==2)
        ix_min = np.where(mask==4)
        
        df_mri.loc[idx, ("Entropy")] =  entropy(img)
        df_mri.loc[idx, ("Entropy Femur")] = entropy(img[ix_fem])
        df_mri.loc[idx, ("Entropy Tibial")] = entropy(img[ix_tib])
        df_mri.loc[idx, ("Entropy Meniscus")] = entropy(img[ix_min])
        
        #print("Done: ", idx, )
        
        if a/5 == a//5:
            print(a)
            df_mri.to_csv("mri_data")

# Ad Opening and closening

Connect

In [17]:
df_mri_shape = pd.read_csv("mri_data_shape", index_col ="Unnamed: 0")
df_mri_ent = pd.read_csv("mri_data_ent", index_col ="Unnamed: 0")
df_mri_ent = df_mri_ent[["n erosions Femur", "Thick Femur X", "Thick Femur Y", "Thick Femur Z", 
            "n erosions Tibial", "Thick Tibial X", "Thick Tibial Y", "Thick Tibial Z", 
            "n erosions Meniscus", "Thick Meniscus X", "Thick Meniscus Y", "Thick Meniscus Z" ]]


df = pd.concat([df_mri, df_mri_shape,df_mri_ent], axis=1, join="inner")
df=df.rename(columns={"Cartilage: 1": "Femur (vol)", "Cartilage: 2": "Tibial (vol)", "Cartilage: 3": "Patella (vol)","Cartilage: 4": "Meniscus (vol)","Medial Tibial":"Medial Tibial (vol)",
                     "Lateral Tibial": "Lateral Tibial (vol)", 'Medial Femur': 'Medial Femur (vol)', 'Lateral Femur':'Lateral Femur (vol)', 'Medial Miniscus':'Medial Miniscus (vol)',
                     'Lateral Miniscus':'Lateral Miniscus (vol)', 'dist_fem_tib':"JWS (approx)", 'Img_intensity_mean':'Img Intensity (mean)', 'Img_intensity_median':'Img Intensity (median)',
                     'int_tib_mean':'Tibial Intensity (mean)','int_tib_median':'Tibial Intensity (median)','int_fem_mean':'Femur Intensity (mean)', 'int_fem_median':'Femur Intensity (median)',
                     'int_min_mean':"Miniscus Intensity (mean)","int_min_median":"Miniscus Intensity (median)", "n erosions Femur":"Erosions Femur (n)", 'n erosions Tibial':"Erosions Tibial (n)",
                     'n erosions Meniscus':"Erosions Meniscus (n)"
                     })

df.to_csv("df_image")

In [18]:
df

,Femur (vol),Tibial (vol),Patella (vol),Meniscus (vol),Medial Tibial (vol),Lateral Tibial (vol),Medial Femur (vol),Lateral Femur (vol),Medial Miniscus (vol),Lateral Miniscus (vol),...,Thick Femur Y,Thick Femur Z,Erosions Tibial (n),Thick Tibial X,Thick Tibial Y,Thick Tibial Z,Erosions Meniscus (n),Thick Meniscus X,Thick Meniscus Y,Thick Meniscus Z
9000798-Left-V00,124138.0,61541.0,3014.0,58123.0,14070.0,47471.0,17503.0,49270.0,28974.0,29149.0,...,10.208717,16.111356,1,7.157595,14.638677,13.926454,6,9.463204,17.391682,16.140794
9000798-Left-V01,99519.0,69283.0,44.0,52372.0,23863.0,45420.0,11896.0,41788.0,22278.0,30094.0,...,9.328740,14.068278,2,8.515610,13.801394,14.653765,6,9.444905,16.615482,17.850034
9000798-Left-V03,103289.0,69378.0,115.0,52494.0,23555.0,45823.0,11688.0,44472.0,22698.0,29796.0,...,9.389909,14.199753,1,8.478309,12.989702,15.735541,2,9.290973,16.787336,17.958946
9000798-Left-V05,93251.0,52852.0,1278.0,48753.0,41173.0,11679.0,26751.0,33046.0,20199.0,28554.0,...,9.057012,14.466491,1,7.327326,14.448332,13.692228,4,9.210845,15.691342,18.453066
9000798-Left-V06,88185.0,62089.0,131.0,46197.0,25434.0,36655.0,7550.0,39146.0,17625.0,28572.0,...,8.632048,12.123316,1,8.684991,11.517158,14.127190,1,9.171531,16.029493,14.935984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9968721-Right-V03,59886.0,32010.0,0.0,25386.0,14751.0,17259.0,18377.0,17506.0,10581.0,14805.0,...,9.726490,11.746960,1,5.300546,26.520298,12.996346,3,6.664741,14.025414,10.211585
9973199-Right-V00,41315.0,30302.0,1505.0,35212.0,14202.0,16100.0,9290.0,18025.0,20796.0,14416.0,...,7.173989,8.322925,1,4.884268,22.800602,10.057086,4,7.643152,17.819838,10.938801
9985803-Left-V03,85246.0,37407.0,43.0,28855.0,17098.0,20309.0,23559.0,26285.0,14167.0,14688.0,...,8.617671,10.622555,1,5.050905,30.536327,12.427575,1,6.794208,18.484946,12.432141
9985803-Right-V03,76106.0,37867.0,2253.0,32270.0,19468.0,18399.0,17301.0,34718.0,17446.0,14824.0,...,7.829835,10.292940,1,5.010851,29.887135,12.266602,4,6.929354,16.347518,11.070326
